In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
import os
import re
from matplotlib.cbook import boxplot_stats
import pickle
sbn.set_context(font_scale=1.5)

In [177]:
# I am going to load the R00 DF to compare our columns
with open ('./Clean Data R00/SURVEY_DF.pickle', 'rb') as handle:
    r00_survey = pickle.load(handle)

In [178]:
#I am goign to read the excel file from DND
dnd_survey = pd.read_excel('../../DND_Complete.xlsx')

In [179]:
dnd_survey.head()

,SubjectID,age,age_group,female,ethnicity,race,part_1_consent,occupation,birthdate,education,...,total_CT,total_gonads,total_urinary_bladder,CT_dosimetry_total_red_marrow,total_red_marrow,adverse_reaction_notes,VANTAGE,part_2_complete,incomplete_part_2_notes,notes
0,DND001,26,0,1,2,5,2015-10-16,Registered Nurse,1989-07-16,"B.S, current graduate student (Nursing)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DND002,50,1,1,2,5,2015-11-02,Disabled,1965-03-15,14 years-high school dropout,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DND003,57,1,1,2,5,2015-10-27,CPA,1958-05-08,MBA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DND004,24,0,1,2,4,2015-11-03,Certified Nurse Aid,1991-06-07,Associates in Medical Assisting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DND005,24,0,1,2,4,2015-11-24,Graduate Research Assistant,1991-04-04,Professiaonl Student-U.E.DD. candidate,...,7.0,10.86024,73.199952,14.7,29.497896,NaN,1.0,1.0,NaN,NaN


In [180]:
#Lets modified all the column name to lowercase just to make comparisions with r00
dnd_survey.columns = [i.lower() for i in dnd_survey.columns]
dnd_survey.columns

Index(['subjectid', 'age', 'age_group', 'female', 'ethnicity', 'race',
       'part_1_consent', 'occupation', 'birthdate', 'education',
       ...
       'total_ct', 'total_gonads', 'total_urinary_bladder',
       'ct_dosimetry_total_red_marrow', 'total_red_marrow',
       'adverse_reaction_notes', 'vantage', 'part_2_complete',
       'incomplete_part_2_notes', 'notes'],
      dtype='object', length=1763)

In [181]:
#I will extract the column name from R00 hoping that they used the same names
cols = {}
for k in r00_survey.keys():
    cols[k] = list(r00_survey[k].columns)

In [182]:
cols.keys()

dict_keys(['Demographics', 'Medical Screening', 'AVI', 'BISBAS', 'BIS', 'TPQ-NS', 'NEO-SF', 'SWLS', 'FTP', 'SBQ'])

###  Let's explore the columns names
#### Demographic

In [183]:
demographic = dnd_survey.filter(items= [i.lower() for i in cols['Demographics']])
demographic

,occupation,education,ethnicity,marital_status,medications
0,Registered Nurse,"B.S, current graduate student (Nursing)",2,Single,"Levonorgestrel, Ethinyl Estradiol"
1,Disabled,14 years-high school dropout,2,Single,"Vitamins, preseriped, goberpentine"
2,CPA,MBA,2,Divorced,NaN
3,Certified Nurse Aid,Associates in Medical Assisting,2,Single,NaN
4,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,2,Married,NaN
...,...,...,...,...,...
117,Artist,"BA from Stanford, JD from Univ of Michigan",2,Married,NaN
118,Associate FedEx,"College, Masters",2,Divorced,NaN
119,Finisher,12th,2,Single,NaN
120,Valet Parking & Rideshare Driver,"High School Diploma, BA, MBA",2,Married,NaN


In [184]:
len ([i.lower() for i in cols['Demographics']]) , len(demographic.columns)

(11, 5)

We can see that we have a discrepancy in the number of columns. We should explore what is the discrepancy for every columns. Next we could try to explore by eye to see if the columns are codded in a different way. This should be really tedious, another alternative is use the variables that have the same name for both experimentsa

In [185]:
def column_finder(colname, df):
    '''
    It's going to provide the column names that matches DND and R00 (everything lower cased) and will return the DND df with these columns
    '''
    df = df.filter(items= [i.lower() for i in cols[colname]])
    diff = len ([i.lower() for i in cols[colname]])  - len(df.columns)
    return diff,df.columns,df

##### Let's check the function works

In [186]:
diff, names, df = column_finder('Demographics',dnd_survey)

In [187]:
diff, names

(6, Index(['occupation', 'education', 'ethnicity', 'marital_status',
        'medications'],
       dtype='object'))

In [188]:
df.head()

,occupation,education,ethnicity,marital_status,medications
0,Registered Nurse,"B.S, current graduate student (Nursing)",2,Single,"Levonorgestrel, Ethinyl Estradiol"
1,Disabled,14 years-high school dropout,2,Single,"Vitamins, preseriped, goberpentine"
2,CPA,MBA,2,Divorced,NaN
3,Certified Nurse Aid,Associates in Medical Assisting,2,Single,NaN
4,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,2,Married,NaN


### It works. Lets do a loop in order to see possible problems

In [189]:
for i in cols.keys(): 
    diff, names, df = column_finder(i,dnd_survey)
    print(f'For the data {i}, the column difference between R00 and DND is {diff} over {len(cols[i])}')

For the data Demographics, the column difference between R00 and DND is 6 over 11
For the data Medical Screening, the column difference between R00 and DND is 88 over 88
For the data AVI, the column difference between R00 and DND is 138 over 138
For the data BISBAS, the column difference between R00 and DND is 45 over 46
For the data BIS, the column difference between R00 and DND is 52 over 53
For the data TPQ-NS, the column difference between R00 and DND is 69 over 74
For the data NEO-SF, the column difference between R00 and DND is 126 over 126
For the data SWLS, the column difference between R00 and DND is 7 over 7
For the data FTP, the column difference between R00 and DND is 15 over 15
For the data SBQ, the column difference between R00 and DND is 20 over 20


This is terrible

---

# SBQ

In [190]:
#SBQ total is not calculated in the DND file, so I will do that now

In [191]:
dnd_survey.filter(regex='(sbq)|(subjectid)').head()

,subjectid,sbq_1,sbq_2,sbq_3,sbq_4,sbq_5,sbq_6,sbq_7,sbq_8,sbq_9,sbq_10,sbq_11,sbq_12,sbq_13,sbq_14,sbq_15,sbq_16,sbq_17,sbq_18
0,DND001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DND002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DND003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DND004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DND005,3.0,3.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,5.0,3.0,2.0,4.0,2.0,4.0,4.0,2.0,2.0


In [192]:
dnd_sbq = dnd_survey.filter(regex='(sbq)|(subjectid)').copy()

In [193]:
dnd_sbq['SBQ'] = dnd_sbq.iloc[:,1:-1].sum(axis=1)

In [194]:
dnd_sbq.columns = ['Subject'] + ['Q'+i[4:] for i in dnd_sbq.columns[1:-1]] + ['SBQ']

In [195]:
dnd_sbq

,Subject,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,SBQ
0,DND001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,DND002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,DND003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,DND004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,DND005,3.0,3.0,4.0,2.0,5.0,3.0,4.0,2.0,4.0,5.0,3.0,2.0,4.0,2.0,4.0,4.0,2.0,2.0,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,DND118,3.0,3.0,4.0,3.0,4.0,2.0,2.0,2.0,4.0,3.0,3.0,4.0,3.0,3.0,3.0,4.0,3.0,3.0,53.0
118,DND119,3.0,3.0,4.0,2.0,4.0,2.0,2.0,4.0,4.0,3.0,3.0,4.0,4.0,3.0,2.0,4.0,1.0,4.0,52.0
119,DND120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
120,DND121,2.0,3.0,4.0,2.0,2.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,4.0,4.0,2.0,55.0


# FTP
FTP : ftp_total

In [196]:
dnd_ftp = dnd_survey.loc[:,['subjectid','ftp_total']]

In [197]:
dnd_ftp = dnd_ftp.rename(columns={'subjectid' : 'Subject',
                'ftp_total' : 'FTP'})
dnd_ftp

,Subject,FTP
0,DND001,NaN
1,DND002,NaN
2,DND003,NaN
3,DND004,NaN
4,DND005,60.0
...,...,...
117,DND118,51.0
118,DND119,54.0
119,DND120,NaN
120,DND121,35.0


# BIS
BIS_total : bis_total

In [198]:
dnd_bis = dnd_survey.loc[:,['subjectid','bis_total']]
dnd_bis = dnd_bis.rename(columns={'subjectid' : 'Subject',
                                  'bis_total' : 'BIS_total'})
dnd_bis


,Subject,BIS_total
0,DND001,NaN
1,DND002,NaN
2,DND003,NaN
3,DND004,NaN
4,DND005,44.0
...,...,...
117,DND118,71.0
118,DND119,61.0
119,DND120,NaN
120,DND121,72.0


# BISBAS

'BAS_D' : BAS_Drive,
'BAS_FS' : BAS_Fun_Seeking,
'BAS_RR' :BAS_Reward_Responsiveness,
'BIS.5' : BIS

In [199]:
bisbas_cols = { 'subjectid' : 'Subject',
                'bas_drive': 'BAS_D'  ,
                'bas_fun_seeking': 'BAS_FS'  ,
                'bas_reward_responsiveness': 'BAS_RR' ,
                'bis' : 'BIS.5'
              }
dnd_bisbas = dnd_survey.loc[:, list(bisbas_cols.keys()) ]

In [200]:
dnd_bisbas = dnd_bisbas.rename(columns=bisbas_cols)
dnd_bisbas.head()

,Subject,BAS_D,BAS_FS,BAS_RR,BIS.5
0,DND001,NaN,NaN,NaN,NaN
1,DND002,NaN,NaN,NaN,NaN
2,DND003,NaN,NaN,NaN,NaN
3,DND004,NaN,NaN,NaN,NaN
4,DND005,9.0,14.0,11.0,23.0


# SWLS
SWLS : swls_total

In [201]:
dnd_swls = dnd_survey.loc[:,['subjectid','swls_total']]
dnd_swls = dnd_swls.rename(columns = {'subjectid' : 'Subject',
                                      'swls_total' : 'SWLS'})
dnd_swls

,Subject,SWLS
0,DND001,NaN
1,DND002,NaN
2,DND003,NaN
3,DND004,NaN
4,DND005,25.0
...,...,...
117,DND118,19.0
118,DND119,23.0
119,DND120,NaN
120,DND121,4.0


# NEO-SF
    
'Conscientiousness': neo_C_total,
'Extraversion':neo_E_total,
'Neuroticism':neo_N_total

In [202]:
neo_cols =  { 'subjectid' : 'Subject',
               'neo_c_total' :'Conscientiousness',
               'neo_e_total' : 'Extraversion',
               'neo_n_total': 'Neuroticism'}

dnd_neo = dnd_survey.loc[:, list(neo_cols.keys()) ]

In [203]:
dnd_neo = dnd_neo.rename(columns= neo_cols)
dnd_neo.head()

,Subject,Conscientiousness,Extraversion,Neuroticism
0,DND001,NaN,NaN,NaN
1,DND002,NaN,NaN,NaN
2,DND003,NaN,NaN,NaN
3,DND004,NaN,NaN,NaN
4,DND005,209.0,171.0,120.0


# TPQ-NS    
 'NS_total' : NS_Total

In [204]:
dnd_tpq = dnd_survey.loc[:,['subjectid','ns_total']]
dnd_tpq = dnd_tpq.rename(columns = {'subjectid' : 'Subject',
                                      'ns_total' : 'NS_total'})
dnd_tpq

,Subject,NS_total
0,DND001,NaN
1,DND002,NaN
2,DND003,NaN
3,DND004,NaN
4,DND005,14.0
...,...,...
117,DND118,17.0
118,DND119,18.0
119,DND120,NaN
120,DND121,16.0


# AVI

HAP_actual : actual_HAP,
P_actual : actual_P,
LAP_actual_LAP,
LA_actual : actual_LA,
LAN_actual : actual_LAN,
N_actual : actual_N,
HAN_actual : actual_HAN,
HA_actual : actual_HA,

HAP_ideal : ideal_HAP,
P_ideal : ideal_P,
LAP_ideal : ideal_LAP,
LA_ideal : ideal_LA,
LAN_ideal : ideal_LAN,
N_ideal : ideal_N,
HAN_ideal : ideal_HAN,
HA_ideal : ideal_HA

In [205]:
avi_cols = {'HAP_actual' : 'actual_hap',
'P_actual' : 'actual_p',
'LAP_actual' : 'actual_lap',
'LA_actual' : 'actual_la',
'LAN_actual' : 'actual_lan',
'N_actual' : 'actual_n',
'HAN_actual' : 'actual_han',
'HA_actual' : 'actual_ha',
'HAP_ideal' : 'ideal_hap',
'P_ideal' : 'ideal_p',
'LAP_ideal' : 'ideal_lap',
'LA_ideal' : 'ideal_la',
'LAN_ideal' : 'ideal_lan',
'N_ideal' : 'ideal_n',
'HAN_ideal' : 'ideal_han',
'HA_ideal' : 'ideal_ha' }
#revert the key values
avi_cols = {v:k for k,v in avi_cols.items()}
dnd_avi = dnd_survey.loc[:, ['subjectid'] + [i.lower() for i in avi_cols.keys()] ]

In [206]:
avi_cols['subjectid'] = 'Subject'
dnd_avi = dnd_avi.rename(columns = avi_cols)

In [207]:
dnd_avi

,Subject,HAP_actual,P_actual,LAP_actual,LA_actual,LAN_actual,N_actual,HAN_actual,HA_actual,HAP_ideal,P_ideal,LAP_ideal,LA_ideal,LAN_ideal,N_ideal,HAN_ideal,HA_ideal
0,DND001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DND002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DND003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DND004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DND005,14.0,12.0,19.0,7.0,7.0,6.0,5.0,8.0,18.0,14.0,21.0,5.0,4.0,3.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,DND118,12.0,7.0,13.0,5.0,6.0,6.0,5.0,7.0,17.0,12.0,16.0,5.0,3.0,3.0,3.0,8.0
118,DND119,12.0,12.0,18.0,8.0,6.0,3.0,3.0,8.0,15.0,12.0,18.0,7.0,5.0,4.0,3.0,10.0
119,DND120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,DND121,16.0,11.0,18.0,8.0,8.0,10.0,6.0,7.0,18.0,15.0,24.0,9.0,4.0,3.0,3.0,9.0


# Demographics

In [208]:
cols['Demographics']

['Subject',
 'Occupation',
 'Education',
 'Ethnicity',
 'Religion',
 'Sex',
 'Marital_Status',
 'Children',
 'Lives_with',
 'Household_income',
 'Medications']

In [209]:
# no religion for DND 
# no children for DND we need to calculate that using children_male  + children_female *
# education is numerical in r00 but categorical in dnd 
# recode female in dnd *
# ethnicity in DND is equal to race in R00 * 
# lives_with could be removed *
# income_household from DND or Household_income from R00 should be recoded *

cols_demo = ['subjectid','occupation', 'education', 'race', 'marital_status','medications', 'age', 'female', 'income_household', 'children_male'  , 'children_female']

In [210]:
dnd_demographics = dnd_survey.filter(items = cols_demo).copy()
dnd_demographics['children'] = dnd_demographics['children_male'] + dnd_demographics['children_female']

In [211]:
dnd_demographics = dnd_demographics.drop(columns= ['children_male'  , 'children_female'])
dnd_demographics = dnd_demographics.rename(columns={'subjectid': 'subject', 'income_household':'household_income'})

In [212]:
dnd_demographics['sex'] = 'Male'
dnd_demographics['sex'].loc[dnd_demographics['female'] == 1] = 'Female'
dnd_demographics = dnd_demographics.drop(columns='female').copy()

C:\Users\joaqu\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [213]:
dnd_demographics.head()

,subject,occupation,education,race,marital_status,medications,age,household_income,children,sex
0,DND001,Registered Nurse,"B.S, current graduate student (Nursing)",5,Single,"Levonorgestrel, Ethinyl Estradiol",26,1.0,0,Female
1,DND002,Disabled,14 years-high school dropout,5,Single,"Vitamins, preseriped, goberpentine",50,2.0,0,Female
2,DND003,CPA,MBA,5,Divorced,NaN,57,8.0,2,Female
3,DND004,Certified Nurse Aid,Associates in Medical Assisting,4,Single,NaN,24,6.0,0,Female
4,DND005,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,4,Married,NaN,24,4.0,0,Female


In [214]:
#Recoding variables
## Race
race = {1:'American Indian/Alaskan Native',
        2:'Asian',
        3:'Native Hawaiian/Pacific Islander',
        4:'Black',
        5:'White',
        6:'More than 1 race',
        7:'I prefer not to answer'}
dnd_demographics['ethnicity'] = dnd_demographics['race'].replace(race)
dnd_demographics.drop(columns='race')

,subject,occupation,education,marital_status,medications,age,household_income,children,sex,ethnicity
0,DND001,Registered Nurse,"B.S, current graduate student (Nursing)",Single,"Levonorgestrel, Ethinyl Estradiol",26,1.0,0,Female,White
1,DND002,Disabled,14 years-high school dropout,Single,"Vitamins, preseriped, goberpentine",50,2.0,0,Female,White
2,DND003,CPA,MBA,Divorced,NaN,57,8.0,2,Female,White
3,DND004,Certified Nurse Aid,Associates in Medical Assisting,Single,NaN,24,6.0,0,Female,Black
4,DND005,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,Married,NaN,24,4.0,0,Female,Black
...,...,...,...,...,...,...,...,...,...,...
117,DND118,Artist,"BA from Stanford, JD from Univ of Michigan",Married,NaN,59,16.0,1,Male,White
118,DND119,Associate FedEx,"College, Masters",Divorced,NaN,57,10.0,1,Male,Black
119,DND120,Finisher,12th,Single,NaN,50,4.0,2,Male,White
120,DND121,Valet Parking & Rideshare Driver,"High School Diploma, BA, MBA",Married,NaN,61,8.0,4,Male,White


In [215]:
#Recoding variables
## Income 
income = {1:'Less than $10,000',
2:'$10,000-$19,999',
3:'$20,000-$29,999',
4:'$30,000-$39,999',
5:'$40,000-$49,999',
6:'$50,000-$59,999',
7:'$60,000-$69,999',
8:'$70,000-$79,999',
9:'$80,000-$89,999',
10:'$90,000-$99,999',
11:'$100,000-$109,999',
12:'$110,000-$119,999',
13:'$120,000-$129,999',
14:'$130,000-$139,000',
15:'$140,000-$149,999',
16:'$150,000 or more '}
dnd_demographics['household_income'] = dnd_demographics['household_income'].replace(income)

In [216]:
#Capitalize all the column name
dnd_demographics.columns = [i.capitalize() for i in dnd_demographics.columns]

In [217]:
dnd_demographics

,Subject,Occupation,Education,Race,Marital_status,Medications,Age,Household_income,Children,Sex,Ethnicity
0,DND001,Registered Nurse,"B.S, current graduate student (Nursing)",5,Single,"Levonorgestrel, Ethinyl Estradiol",26,"Less than $10,000",0,Female,White
1,DND002,Disabled,14 years-high school dropout,5,Single,"Vitamins, preseriped, goberpentine",50,"$10,000-$19,999",0,Female,White
2,DND003,CPA,MBA,5,Divorced,NaN,57,"$70,000-$79,999",2,Female,White
3,DND004,Certified Nurse Aid,Associates in Medical Assisting,4,Single,NaN,24,"$50,000-$59,999",0,Female,Black
4,DND005,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,4,Married,NaN,24,"$30,000-$39,999",0,Female,Black
...,...,...,...,...,...,...,...,...,...,...,...
117,DND118,Artist,"BA from Stanford, JD from Univ of Michigan",5,Married,NaN,59,"$150,000 or more",1,Male,White
118,DND119,Associate FedEx,"College, Masters",4,Divorced,NaN,57,"$90,000-$99,999",1,Male,Black
119,DND120,Finisher,12th,5,Single,NaN,50,"$30,000-$39,999",2,Male,White
120,DND121,Valet Parking & Rideshare Driver,"High School Diploma, BA, MBA",5,Married,NaN,61,"$70,000-$79,999",4,Male,White


## To do
Decide what to do with medical screenings [x]

Join all the df in a single df

Add the age column to R00_survey['Demographics'] and calculate BMI for R00_survey [x]

### medical screenings

In [218]:
#there is no heigh in DND but i can calculate that using bmi, also there is no BMI in R00 
    
medic_cols =  { 'subjectid' : 'Subject', 
               'bmi' : 'BMI',
                'weight' : 'Weight (kg)',
                'height': 'Height (cm)'}

r00_med = r00_survey['Medical Screening'].filter( items =  [i for i in medic_cols.values()]).copy()
r00_med['BMI'] = r00_med['Weight (kg)'] / (r00_med['Height (cm)']/100)**2  # weight/height^2 (height in meters)
#Adding BMI to R00
r00_survey['Medical Screening']['BMI'] = r00_med['BMI']
r00_survey['Medical Screening'].head()

,Subject,Med History,Height (cm),Weight (kg),Vitals: Temp (˚F),Vitals: Pulse,Vitals: Resp/min,Vitals: BP,Allergies:,Physical: General Appearance,...,Toxicology: Benzodiazepines,Toxicology: Cocaine,Toxicology: Opiates,Toxicology: Oxycodone,Toxicology: Phencyclidine,Toxicology: Methadone,MRI Eligibility?,PET Eligbility?,Notes/Specific Abnormalities:,BMI
1,1002,Psychiatric disorder-P. Grandmother,161.0,61.5,98.0,73.0,16.0,117/84,Penicillin,Normal,...,Negative,Negative,Negative,Negative,Negative,Negative,Yes,Yes,NaN,23.725936
3,1004,Paxil-2014 (2 month usage),174.0,91.5,97.9,50.0,20.0,132/73,None,Normal,...,Negative,Negative,Negative,Negative,Negative,Negative,Yes,Yes,NaN,30.221958
7,1008,"Current: EPAP apnea, RX-20mg acifex, 1995- her...",182.0,103.7,97.2,77.0,16.0,118/72,None,Normal,...,Negative,Negative,Negative,Negative,Negative,Negative,Yes,Yes,NaN,31.306605
8,1009,None,175.0,87.0,96.7,54.0,16.0,118/70,None,Normal,...,Negative,Negative,Negative,Negative,Negative,Negative,Yes,Yes,NaN,28.408163
13,1014,"HBP, high cholesterol, daughter-MDD, father-MI...",167.6,87.8,98.4,100.0,16.0,111/80,None,Normal,...,Negative,Negative,Negative,Negative,Negative,Negative,Yes,Yes,NaN,31.256942


In [219]:
dnd_medi = dnd_survey.loc[:,[i for i in medic_cols.keys() if i in dnd_survey.columns]]

In [220]:
dnd_medi['height'] =  np.sqrt(dnd_medi['weight'] / dnd_medi['bmi'])*100

In [221]:
dnd_medi = dnd_medi.rename(columns= medic_cols)
dnd_medi

,Subject,BMI,Weight (kg),Height (cm)
0,DND001,26.2,NaN,NaN
1,DND002,16.0,NaN,NaN
2,DND003,30.4,NaN,NaN
3,DND004,39.1,NaN,NaN
4,DND005,21.9,63.5,170.280456
...,...,...,...,...
117,DND118,22.9,72.5,177.930854
118,DND119,31.8,97.8,175.370228
119,DND120,NaN,NaN,NaN
120,DND121,34.2,114.4,182.894211


# Adding age to R00 

In [222]:
r00_overview = pd.read_excel('../../R00_Survey.xlsx', sheet_name= 'Overview')

In [223]:
r00_overview = r00_overview.filter(items = ['Subject','Age']).copy()

In [224]:
r00_survey['Demographics'] = r00_survey['Demographics'].merge(r00_overview, on = 'Subject')

In [225]:
r00_survey['Demographics'].head()

,Subject,Occupation,Education,Ethnicity,Religion,Sex,Marital_Status,Children,Lives_with,Household_income,Medications,Age
0,1002,Restaurant worker,14.0,White,Buddhism,Female,Single,0.0,"Mother, father, two younger brothers","$10,000-$19,999",None,32.0
1,1004,Student/Coach,14.0,White/Pacific Islander,Questionable,Male,Single,0.0,Grandparents,"$10,000-$19,999",None,28.0
2,1008,IT Manager,16.0,White,Judiasm,Male,Married,2.0,"Wife, two children","$150,000 or more",Aciphex,46.0
3,1009,Professor,22.0,White,Judiasm,Male,Married,0.0,Spouse/partner,"$150,000 or more",None,58.0
4,1014,Realtor,18.0,White,Christianity,Female,Married,3.0,Spouse/partner,"$120,000-$129,999","Lipitor, Diovan, 81mg Aspirin",68.0


In [226]:
r00_survey['Demographics'].shape

(36, 12)

In [137]:
#Saving the complete df
with open('./Clean Data R00/SURVEY_DF_age_bmi.pickle', 'wb') as handle:
    pickle.dump(r00_survey, handle)

----

In [227]:
#Join all df in a structured DF
cols.keys()

dict_keys(['Demographics', 'Medical Screening', 'AVI', 'BISBAS', 'BIS', 'TPQ-NS', 'NEO-SF', 'SWLS', 'FTP', 'SBQ'])

In [228]:
dnd_all = {'Demographics' : dnd_demographics,
           'Medical Screening': dnd_medi,
           'AVI' : dnd_avi,
           'BISBAS' : dnd_bisbas,
           'BIS' : dnd_bis,
           'TPQ-NS' : dnd_tpq,
           'NEO-SF' : dnd_neo,
           'SWLS' :  dnd_swls,
           'FTP' : dnd_ftp,
           'SBQ' : dnd_sbq}

In [229]:
from collections import OrderedDict
dnd_all = OrderedDict(dnd_all)
dnd_all.keys()

odict_keys(['Demographics', 'Medical Screening', 'AVI', 'BISBAS', 'BIS', 'TPQ-NS', 'NEO-SF', 'SWLS', 'FTP', 'SBQ'])

In [259]:
#Saving the complete DND df
with open('./Clean_DND/SURVEY_DND.pickle', 'wb') as handle:
    pickle.dump(dnd_all, handle)

In [260]:
#Testing if is working
with open('./Clean_DND/SURVEY_DND.pickle','rb') as DND:
        DND_df = pickle.load(DND)

In [262]:
DND_df['Demographics'].head()

,Subject,Occupation,Education,Race,Marital_status,Medications,Age,Household_income,Children,Sex,Ethnicity
0,DND001,Registered Nurse,"B.S, current graduate student (Nursing)",5,Single,"Levonorgestrel, Ethinyl Estradiol",26,"Less than $10,000",0,Female,White
1,DND002,Disabled,14 years-high school dropout,5,Single,"Vitamins, preseriped, goberpentine",50,"$10,000-$19,999",0,Female,White
2,DND003,CPA,MBA,5,Divorced,NaN,57,"$70,000-$79,999",2,Female,White
3,DND004,Certified Nurse Aid,Associates in Medical Assisting,4,Single,NaN,24,"$50,000-$59,999",0,Female,Black
4,DND005,Graduate Research Assistant,Professiaonl Student-U.E.DD. candidate,4,Married,NaN,24,"$30,000-$39,999",0,Female,Black
